In [1]:
import pandas as pd

# Tratamento da base

In [3]:
crise_df = pd.read_excel("../data/global_crisis_data.xlsx")

In [4]:
cols_to_drop = [
    "Case", "CC3",  'exch_usd', 'exch_usd_alt1', 'exch_usd_alt2', 'exch_usd_alt3', 'conversion_notes',
    'national currency', "exch_primary source code", "exch_sources", 'GDP_Weighted_default', '<',
    'Independence', 'Gold Standard'
]

cols_to_rename = {
    "Country" : "country_name",
    "Year" : "ref_year",
    'Banking Crisis ' : "crisis_banking",
    'Banking_Crisis_Notes' : "notes_crisis_banking", 
    'Systemic Crisis' : "crisis_systemic",
    'Domestic_Debt_In_Default' : "default_domestic_debt", 
    'Domestic_Debt_ Notes/Sources' : "notes_default_domestic_debt",
    'SOVEREIGN EXTERNAL DEBT 1: DEFAULT and RESTRUCTURINGS, 1800-2012--Does not include defaults on WWI debt to United States and United Kingdom and post-1975 defaults on Official External Creditors' : "default_sovereign_debt_tp1",
    'SOVEREIGN EXTERNAL DEBT 2: DEFAULT and RESTRUCTURINGS, 1800-2012--Does not include defaults on WWI debt to United States and United Kingdom but includes post-1975 defaults on Official External Creditors' : "default_sovereign_debt_tp2",
    'Defaults_External_Notes' : "notes_default_sovereign_debt", 
    'Inflation, Annual percentages of average consumer prices' : "ref_inflation",
    'Currency Crises' : "crisis_currency",
    'Inflation Crises' : "crisis_inflation"
}

cols_new_types = {x : bool for x in cols_to_rename.values() if x.startswith(("crisis", "default"))}
cols_new_types["ref_year"] = int

crise_df = (
    crise_df[1:]
    .drop(cols_to_drop, axis=1)
    .rename(cols_to_rename, axis=1)
    .astype(cols_new_types)
)

Mantendo apenas valores presentes antes

In [5]:
crise_df = crise_df[crise_df["ref_year"].between(1946, 2009)]

In [6]:
crise_df.head(3)

,country_name,ref_year,crisis_banking,notes_crisis_banking,crisis_systemic,default_domestic_debt,notes_default_domestic_debt,default_sovereign_debt_tp1,default_sovereign_debt_tp2,notes_default_sovereign_debt,ref_inflation,crisis_currency,crisis_inflation
147,Algeria,1946,False,NaN,False,False,NaN,False,False,NaN,18.181818,False,False
148,Algeria,1947,False,NaN,False,False,NaN,False,False,NaN,69.230769,False,True
149,Algeria,1948,False,NaN,False,False,NaN,False,False,NaN,63.636364,False,True


# Exploração da base

## AutoEDA

In [50]:
import pandas_profiling

report = pandas_profiling.ProfileReport(crise_df)
report.to_file("../reports/raw_crisis.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 114.95it/s]


## Tratamentos específicos

#### Tratando `ref_inflation`

Transformaremos os `"  "` de `ref_inflation` em NAs para podermos converter para *float*.

In [7]:
import numpy as np

In [8]:
crise_df["ref_inflation"] = [np.nan if x in [" ", "  "] else x for x in crise_df["ref_inflation"]]

In [9]:
crise_df["ref_inflation"] = pd.to_numeric(crise_df["ref_inflation"])

#### Capitalizando variáveis

In [10]:
for cols in crise_df.columns:
    if crise_df.dtypes[cols] == 'O':
        crise_df[cols] = crise_df[cols].str.upper()

#### Variáveis explicativas

In [11]:
crise_df["notes_default_sovereign_debt"].value_counts()

OIL AND INTEREST RATE SHOCKS                               106
DEPRESSION AND WORLD WAR II                                 32
DEPRESSION, WORLD WAR II. AND COMMUNIST TAKEOVER            22
REPUDIATION FOLLOWING INDEPENDENCE                          16
SHARP EXPORTS CONTRACTION. OIL AND INTEREST RATE SHOCKS     15
DEPRESSION                                                  13
SOVIET COLLAPSE, OIL AND INTEREST RATE SHACKS               12
OIL AND INTEREST RATE SHOCKS. BUDGET CRISIS                 12
OIL AND INTENST RATE SHOCKS                                 12
CIVIL UNREST, THEN DEPRESSION                               10
OIL AND INMEST RATE SHACKS                                  10
OIL AND INTEREST RATE SHOCKS. NATURAL DISASTERS             10
POST-PERON BUDGET CRISIS. BEET EXPORT DROPS                 10
OIL AND INTEREST RATE SHOCKS; SOVIET COLLAPSE               10
OIL AND INTEREST RATE SHOCKS AND BUDGET CRISIS              10
INTEREST RATE SHOCKS                                   

Das variáveis explicativas (`notes_[...]`), manteremos apenas a `notes_default_sovereign_debt`, que apresenta baixa cardinalidade e valores fáceis de agrupar (serão criados quatro grupos: `oil`, `war`, `rate` e `others`; pode-se ser atribuído a mais de um). 

In [12]:
notes_cols_to_expande = ["OIL", "WAR", "RATE"]

for cols in notes_cols_to_expande:
    crise_df[f"notes_default_sovereign_debt_{cols}"] = crise_df["notes_default_sovereign_debt"].str.contains(cols)

In [13]:
crise_df["notes_default_sovereign_debt_OTHERS"] = (~crise_df["notes_default_sovereign_debt"].isna()) & (~(crise_df["notes_default_sovereign_debt_OIL"] | crise_df["notes_default_sovereign_debt_WAR"] | crise_df["notes_default_sovereign_debt_RATE"]))

Manteremos `default_sovereign_debt_tp2` em detrimento da `default_sovereign_debt_tp1` (há mudança em apenas oito ocorrências, e a segunda é mais compreensiva).

In [14]:
crise_df = (
    crise_df
    .drop(["notes_default_domestic_debt", "notes_crisis_banking", "default_sovereign_debt_tp1"], axis=1)
)

In [15]:
crise_df.isnull().sum()

country_name                              0
ref_year                                  0
crisis_banking                            0
crisis_systemic                           0
default_domestic_debt                     0
default_sovereign_debt_tp2                0
notes_default_sovereign_debt           4085
ref_inflation                           205
crisis_currency                           0
crisis_inflation                          0
notes_default_sovereign_debt_OIL       4085
notes_default_sovereign_debt_WAR       4085
notes_default_sovereign_debt_RATE      4085
notes_default_sovereign_debt_OTHERS       0
dtype: int64

In [16]:
(crise_df["default_sovereign_debt_tp2"] == 0).sum()

3632

Devido à grande quantidade de valores nulos em `notes_default_sovereign_debt`, ela será removida

In [17]:
crise_df = (
    crise_df
    .drop([x for x in crise_df.columns if x.startswith("notes")], axis=1)
)

Os valores nulos em `ref_inflation` serão tratados em outro notebook

# Salvando arquivo

In [18]:
crise_df.to_csv("../data/preprocessed/crisis_data.csv", index=False)